In [ ]:
%pip install matplotlib seaborn matplotlib-venn pandas

In [ ]:
import pandas as pd
# Load the CSV file (adjust filename/path accordingly)
df = pd.read_csv('Data/1000 Sales Records.csv')

# Show the first 3 rows
df.head(3)
